In [1]:
import torch
import torch.nn.functional as Func
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import os
import copy

In [2]:
class MYCNN(nn.Module):
    def __init__(self):
        super().__init__() 
        #convolution layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels =32, padding=1, kernel_size=3) 
        self.pool1 = nn.MaxPool2d(2) 
        self.conv2 = nn.Conv2d(in_channels=32, out_channels =64, padding=1, kernel_size=3)
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels =128, padding=1, kernel_size=3)
        self.pool3 = nn.MaxPool2d(2) 
        #fully connected layer
        self.fc1 = nn.Linear(128*28*28, 512)
        #dropout to reduce overfitting
        self.drop = nn.Dropout(0.2) 
        #output 5 classes
        self.out = nn.Linear(512, 5)

    def forward(self, x):
        x=x
        x=self.conv1(x)
        x=Func.relu(x)
        x=self.pool1(x)
        x=self.conv2(x)
        x=Func.relu(x)
        x=self.pool2(x)
        x = self.drop(x)  
        x=self.conv3(x)
        x=Func.relu(x)
        x=self.pool3(x)
        
        x = x.view(-1, 128*28*28) 
        x = Func.relu(self.fc1(x))
        x = self.drop(x)
        x = self.out(x) 

        return Func.softmax(x, dim=1)

In [3]:
model = MYCNN()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.load_state_dict(torch.load('handmodel_pytorch_second.pt'))

<All keys matched successfully>

In [43]:
ROOT_DIR2 = 'F://Handproject//test_immage//'
BATCH_SIZE = 1
mean_nums = [0.485, 0.456, 0.406]
std_nums = [0.229, 0.224, 0.225]

test_transforms = transforms.Compose([
                                      transforms.Resize((224, 224)),
                                      transforms.ToTensor(),
                                      #transforms.Normalize(mean_nums, std_nums),
                                      #transforms.RandomRotation(degrees=15),
                                       #transforms.RandomHorizontalFlip(),
                                      ])

test_datasets = datasets.ImageFolder(ROOT_DIR2, test_transforms)
test_dataloaders =torch.utils.data.DataLoader(test_datasets, batch_size=BATCH_SIZE,
                                             shuffle=True)
dataset_sizes = len(test_datasets)


In [45]:
model.eval()
with torch.no_grad():
    for inputs,labels in test_dataloaders:
        #Store in GPU

        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model.forward(inputs)
        _, preds = torch.max(outputs, 1)
        print(f"Original Class: {labels+1}")
        print(f"Prediction : {preds+1}")

Original Class: tensor([3], device='cuda:0')
Prediction : tensor([3], device='cuda:0')
Original Class: tensor([4], device='cuda:0')
Prediction : tensor([4], device='cuda:0')
Original Class: tensor([5], device='cuda:0')
Prediction : tensor([5], device='cuda:0')
Original Class: tensor([2], device='cuda:0')
Prediction : tensor([2], device='cuda:0')
Original Class: tensor([1], device='cuda:0')
Prediction : tensor([1], device='cuda:0')
Original Class: tensor([3], device='cuda:0')
Prediction : tensor([3], device='cuda:0')
